# Curso básico de Python aplicado à Astronomia
### Laboratório Interinstitucional de e-Astronomia
# Aula XX - Relação Massa-riqueza de Aglomerados de Galáxias
Michel Aguena, LAPP/IN2P3 & LIneA

## Objetivo


Aglomerados de galáxias podem ser ferramentas poderosas para se obter infomação sobre a cosmologia,
mas é necessário fazer a associação entre os aglomerados e os halos de matéria escura. Dentre essas propriedades, está a relação entre a riqueza de aglomerados e a massa dos halos.

## Índice
1. [Aglomerados de galáxias](#cluster)
2. [Calibrando uma relação (fazendo um "fit")](#fit)
3. [O espaço de parâmetros](#param)
4. [Calibrando a relação massa-riqueza](#mr)

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
# computation
import numpy as np
from astropy.table import Table
# display
from IPython.display import Markdown as md
from IPython.display import display, Math
# plots
import pylab as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import MultipleLocator
# widgets plot
import ipywidgets as widgets
%matplotlib widget
# auto reload local modules
%load_ext autoreload
%autoreload 2
# my functions
import aux_funcs

# 1. Aglomerados de galáxias <a class="anchor" id="cluster"></a>

Conjuntos de galaxias gravitacionalmente conectadas, composta de centenas a milhares de galaxias e com uma massa da ordem de 100 Trilhões de masssas solares. Podem ser detectados através de emissões em raio-x ($\sim10^{10}$ GHz), pelo efeito Sunyaev Zel'dovich ($\sim100$ GHz), ou em surveys opticos ($\sim10^5$ GHz) de galaxias.
<table>
    <tr>
        <td style="text-align:center"><img src='figs/xray.jpeg' width='200' height='200'/><br>
            Aglomerado Abell 1689 com emissão em <b>raio-x</b>, imagem composta da combinação de observações do HST (Hubble Space Telescope) e CXO (Chandra X-ray Observatory).</td>
        <td style="text-align:center"><img src='figs/sz.jpg' width='220' height='200'/><br>
            Efeito <b>Sunyaev–Zeldovich</b> termico no aglomerado RX J1347.5-1145 observado pelo telescopio ALMA (Atacama Large Millimeter Array).</td>
        <td style="text-align:center"><a href='https://doi.org/10.1093/mnras/stab264'><img src='figs/wazp.jpg' width='240' height='200'/><br>
            Aglomerado <b>optico</b> detectado pelo algoritimo WaZP (Wavelet Z Photometric) no primeiro ano de dados do DES (Dark Energy Survey).</a></td>
    </tr>
</table>

## 1.1 Aplicação em cosmologia

Halos de matréria escura são as maiores estruturas ligadas gravitacionalmente conectadas.
A abundância de halos esta conectada com algumas componentes fundamentais do Universo como a quantidade de matéria escura, de energia escura e sua evolução. Abaixo podemos ver explicitamente como a fração de matéria no Universo ($\Omega_m$) e o parâmetro da evolução da energia escura ($w$) afetam a contagem esperada de halos em cada intervalo de massa e redshift:

In [ ]:
from aux_funcs import plot_nc
%matplotlib widget
plot_nc()

Aglomerados de Galáxias são traçadores dos halos de matéria escura, e podem portanto serem usados para se inferir a abundancia de halos. Porem é necessário compreender o mapeamento entre os halos de materia escura e os aglomerados de galaxias.

Portanto, para que aglomerados sejam usados para se vincular cosmologia, é preciso:

* Detectar os aglomerados
* Caraterizar a detecção (contagem de aglomerados->contagem de halos)
* Caraterizar os redshifts dos aglomerados
* **Determinar um indicativo de massa para os aglomerados**

## 1.2 Relação massa-riqueza


A contagem de halos é extremamente sensível a massa mímima considerada. Isso pode ser visto abaixo:

<table><tr>
    <td><img src='dndlogM.png'/></td>
    <td style="text-align:left">
        Contagem de halos de matéria escura previstos em 10,000 graus$^2$,
        <br> observando-se até redshift 2.
        <table style="border: 1px solid black">
            <tr><th>Massa mínima</th><th>Numero de halos</th></tr>
            <tr><td>$4.0\times10^{13}$</td><td>$\sim800,000$</td></tr>
            <tr><td>$6.3\times10^{13}$</td><td>$\sim450,000$</td></tr>
            <tr><td>$1.0\times10^{14}$</td><td>$\sim230,000$</td></tr>
            <tr><td>$1.6\times10^{14}$</td><td>$\sim110,000$</td></tr>
            <tr><td>$2.5\times10^{14}$</td><td>$\sim50,000$</td></tr>
        </table>
    </td>
</tr></table>

Portanto, é muito importante que haja um mapeamento da massa dos halos com alguma propriedade dos aglomerados.
No caso de aglomerados opticos, essa propriedade é a riqueza do aglomerado, que está associada com o numero de galaxias-membro do aglomerado.

<table><tr>
    <td><a href='https://academic.oup.com/mnras/article/482/1/1352/5123719'><img src='mr_mcclintock.png' width='300'/>McClintock et al. 2018</a></td>
    <td><a href='https://arxiv.org/abs/1910.13548'><img src='mr_kirby.png' width='300'/>Kirby et al. 2019</a></td>
</tr></table>

# 2. Calibrando uma relação (fazendo um "fit") <a class="anchor" id="fit"></a>
Como definir qual os valores de uma função se ajustam melhor um conjunto de dados?
Ex:

In [ ]:
x = np.array([1, 2, 3, 4, 5])
y = np.array([3, 5, 7, 9, 11])
err = np.array([3.12, 7.40, 5.61, 1.48 , 3.50])

In [ ]:
%matplotlib inline
plt.errorbar(x, y, err, ls='', fmt='.', capsize=3)
plt.grid()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Qual a melhor função que descreve esses dados?
Uma reta:
\begin{equation}
f(x) = a\; x + b
\end{equation}

In [ ]:
def func(x, a, b):
    return a*x+b

Que tal testar alguns parâmetros:

In [ ]:
%matplotlib widget
aux_funcs.plot_with_line(x, y, err, show_chi2=False)

In [ ]:
parametros = [(0, 7), (1, 2), (3, 0),
              (2.5, 1), (2, 6), (-3, 15)]
md('Dados alguns parâmetros:\n\n'+
   'Conjunto | a | b \n---|---|---\n'+
    '\n'.join([f'$c_{i}$ | {a} | {b}' for i, (a, b) in enumerate(parametros)])+
   '\n\nQual se ajusta melhor?'
  )

In [ ]:
%matplotlib inline
plt.errorbar(x, y, err, ls='', fmt='.', capsize=3)
for i, (a, b) in enumerate(parametros):
    plt.plot(x, func(x, a, b),
            zorder=0, label=f'$c_{i}$ (a={a}, b={b})')
plt.grid()
plt.legend(ncol=2)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Como avaliar quantativamente qual o melhor ajuste?

## Método do $\chi^2$:

\begin{equation}
\chi^2 = \sum_i \frac{(data_i-modelo_i)^2}{(erro_i)^2}
\end{equation}

In [ ]:
def chi2(data, modelo, erro):
    return sum((data-modelo)**2/erro**2)

In [ ]:
aux_funcs.show_chi2_numbers(x, y, err, parametros, func=func)

In [ ]:
%matplotlib inline
plt.errorbar(x, y, err, ls='', fmt='.', capsize=3)
for i, (a, b) in enumerate(parametros):
    plt.plot(x, func(x, a, b),
            zorder=0, label=f'$c_{i}$ ($\chi^2={chi2(y, func(x, a, b), err):.1f}$)')
plt.grid()
plt.legend(ncol=2)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
%matplotlib widget
aux_funcs.plot_with_line(x, y, err, func=func)

# 3. O espaço de parâmetros <a class="anchor" id="param"></a>

Avaliar a qualidade do ajuste no espaço de parâmetros

In [ ]:
%matplotlib inline
for i, (a, b) in enumerate(parametros):
    plt.scatter(a, b, color=f'C{i}')
    plt.text(a, b, f'$c_{i}$({chi2(y, func(x, a, b), err):.0f})')
plt.xlabel('a')
plt.ylabel('b')
plt.grid()
plt.show()

E se calculassemos os valores na grade?

Definir  valores para a grade

In [ ]:
a_vals = np.linspace(-3, 7, 101)
b_vals = np.linspace(-14, 16, 99)
a_grid, b_grid, chi2_grid = aux_funcs.compute_chi2_grid(a_vals, b_vals, func, x, y, err)

* Grafico 3D do $\chi^2$:

In [ ]:
%matplotlib widget
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(a_grid, b_grid, chi2_grid,
                 lw=.5)

for i, (a, b) in enumerate(parametros): 
    ax.scatter(a, b, chi2(y, func(x, a, b), err),
               color=f'C{i}', label=f'$c_{i}$')

ax.set_xlabel('a')
ax.set_ylabel('b')
ax.set_zlabel('$\chi^2$')

ax.legend(ncol=2)

fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.resizable = True

plt.show()

## Likelihood

A *likelihood* é como uma probablilidade no espaço de parâmetros. No caso de uma distribuição Gaussiana de dados, ela pode ser definida em termos do $\chi^2$:

\begin{equation}
\mathcal{L} = \frac{1}{\sqrt{2\pi \det({erro}^2)}} \exp{\left(-\frac{1}{2}\chi^2\right)}
=\frac{1}{\prod_i\sqrt{2\pi\,{erro}^2_i}}\exp{\left(-\frac{1}{2}\sum_i \frac{(data_i-modelo_i)^2}{(erro_i)^2}\right)}
\end{equation}

In [ ]:
def like_chi2(chi2_, err):
    return np.exp(-.5*chi2_)/np.prod(np.sqrt(2*np.pi)*err)
def like(data, modelo, erro):
    return like_chi2(chi2(data, modelo, erro), erro)

In [ ]:
md('Para o nosso conjunto de dados, temos:\n\n'+
   'Conjunto | $\chi^2$ | $\mathcal{L}$\n---|---|---\n'+
    '\n'.join([f'c{i}|${chi2(y, func(x, a, b), err):.0f}$|'+
                f'${np.exp(-chi2(y, func(x, a, b), err)/2):.2e}$'
               for i, (a, b) in enumerate(parametros)]))

* Grafico 3D da $\mathcal{L}$:

In [ ]:
%matplotlib widget

like_grid = like_chi2(chi2_grid, err)

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(a_grid, b_grid, like_grid,
                 lw=.5)

ax.set_xlabel('a')
ax.set_ylabel('b')
ax.set_zlabel('$\mathcal{L}$')

for i, (a, b) in enumerate(parametros): 
    ax.scatter(a, b, like(y, func(x, a, b), err),
               color=f'C{i}', label=f'c{i}')


fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.resizable = True

plt.show()

## Fixar *VS* marginalizar parâmetros
Como encontrar definitivamente qual conjunto de parâmetros que melhor se ajusta aos dados?

### Fixar parâmetros

Deteminar a likelihood de um parâmetro fixando-se o valor do outro parâmetro:

\begin{equation}
P_f(a) = P(a, b=b_0)
\end{equation}

O que acontece quando se fixa um dos parâmetros:

In [ ]:
%matplotlib inline
f, axes = plt.subplots(1, 2, figsize=(8, 4))

vals = np.linspace(0, 5, 100)
for b in [-4, -2, 0, 2, 4]:
    axes[0].plot(vals, [like(y, func(x, a, b), err) for a in vals],
            label=f'b={b}')

vals = np.linspace(-7, 10, 100)
for a in [1, 1.5, 2, 2.5, 3]:
    axes[1].plot(vals, [like(y, func(x, a, b), err) for b in vals],
            label=f'a={a}')
for ax in axes:
    ax.legend()
    ax.grid()
    ax.set_ylim(0, ax.get_ylim()[1])
axes[0].set_xlabel('a')
axes[1].set_xlabel('b')
axes[1].yaxis.tick_right()
plt.show()

In [ ]:
%matplotlib widget
aux_funcs.plot_like_marg(x, y, err, a_vals, b_vals)

## Marginalizar parâmetros

Deteminar a likelihood de um parâmetro considerando-se todos os valores possíveis que o outro parâmetro pode assumir: 
\begin{equation}
P_m(a) = \int_{-\infty}^{\infty} db P(a, b)
\approx \sum_i \Delta b P(a, b=b_i)
\end{equation}

Vamos comparar as likelihoods marginalizadas com as fixas na seção anterior:

In [ ]:
%matplotlib inline
f, axes = plt.subplots(2, figsize=(6, 9))

norm = lambda x, dx: np.array(x)/sum(np.array(x)*dx)

da, db = a_vals[1]-a_vals[0], b_vals[1]-b_vals[0]
axes[0].plot(a_vals, norm(np.sum(like_grid, axis=1), da), c='0', label='Marginalized')
axes[1].plot(b_vals, norm(np.sum(like_grid, axis=0), db), c='0', label='Marginalized')

axes[0].set_xlabel('a')
axes[0].xaxis.tick_top()
axes[0].xaxis.set_label_position('top') 

axes[1].set_xlabel('b')



for b in [-4, -2, 0, 2, 4]:
    vals = np.linspace(a_vals[0], a_vals[-1], 200)
    dv = (vals[1]-vals[0])
    axes[0].plot(vals, norm([like(y, func(x, a, b), err) for a in vals], dv),
                label=f'b={b}', zorder=0, lw=.8)
for a in [1, 1.5, 2, 2.5, 3]:
    vals = np.linspace(b_vals[0], b_vals[-1], 200)
    dv = (vals[1]-vals[0])
    axes[1].plot(vals, norm([like(y, func(x, a, b), err) for b in vals], dv),
                label=f'a={a}', zorder=0, lw=.8)
    
for ax in axes:
    ax.grid()
    ax.grid(which='minor', lw=.5)
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.legend()
axes[0].set_xlim(-1, 5)
axes[1].set_xlim(-10, 10)
axes[1].xaxis.set_major_locator(MultipleLocator(5))
plt.show()

### Degeneressencia

Há alguma correlação entre os parâmetros calibrados? O valor de um dos dois parâmetros influencia no valor do outro?


Vamos ver a dependencia da likelihood:

In [ ]:
%matplotlib inline
print(like_grid.min(), like_grid.max())
fig = plt.figure(figsize=(10, 7))
cb = plt.contourf(a_grid, b_grid, like_grid,
            levels=np.linspace(like_grid.min(), like_grid.max(), 100)
            )
plt.colorbar(cb)
plt.xlabel('a')
plt.ylabel('b')

Quais valores se ajustam melhor?

Qual a influência da barra de erro dos dados?

E se tivessemos menos dados?

In [ ]:
%matplotlib widget
aux_funcs.plot_fit_degeneressence(x, y, err, a_vals, b_vals)

A correlação entre os parâmetros se dá pelo envolvimento na likelihood.

Ex:

\begin{equation}
\mathcal{L} \propto \exp{\left[-(a^2+b^2+2\theta ab)\right]}
\end{equation}

In [ ]:
%matplotlib widget
aux_funcs.plot_like_deg()

# 4. Calibrando a relação massa-riqueza <a class="anchor" id="mr"></a>

A relação entre a massa dos halos e a riqueza dos aglomerados é approximada por uma relação de escala:

\begin{equation}
\left(\frac{M}{M^0}\right) \approx
\left(\frac{N_{gals}}{N_{gals}^0}\right)^\alpha 
\end{equation}

Abaixo temos um conjunto de dados com a massa, erro e riqueza de alguns aglomerados:

In [ ]:
data = Table(np.genfromtxt('data/MR.dat', delimiter=',', names=True))
data

Visualizar os dados:

In [ ]:
%matplotlib widget
aux_funcs.plot_mr(data)

### A relação massa-riqueza definida em escala logaritimica:

\begin{equation}
logM=\alpha\,logN_{gals}+\beta,
\end{equation}

onde $\beta= logM_0-\alpha\,logN_{gals}^0$.

* Detalhe:
\begin{equation}
Err_{logM}=\frac{Err_{M}}{M ln(10)}
\end{equation}

In [ ]:
%matplotlib widget
aux_funcs.plot_mr(data, fit=True)

## Determinar quem são as quantidades a serem fitadas

In [ ]:
# Preencher:
logN = # ???
logM = # ???
siglogM = # ???

## E os valores dos parâmetros a serem explorados:

In [ ]:
%matplotlib widget
aux_funcs.plot_mr(data, fit=True)

In [ ]:
# Preencher:
alpha_vals = # ???
beta_vals = # ???

### Visualizar em 2D

In [ ]:
%matplotlib inline
aux_funcs.plot_likelihood2D(
    alpha_vals=alpha_vals, beta_vals=beta_vals,
    func=func, x=logN, y=logM, err=siglogM
)

### Ou 3D

In [ ]:
%matplotlib widget
aux_funcs.plot_likelihood3D(
    alpha_vals=alpha_vals, beta_vals=beta_vals,
    func=func, x=logN, y=logM, err=siglogM
)